In [0]:
%pip install bs4

In [0]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import shutil
import urllib.request

In [0]:
credential_df = spark.sql("SHOW STORAGE CREDENTIALS")

In [0]:
credential_df = spark.sql("SHOW STORAGE CREDENTIALS")
unity_storage_credential = credential_df.collect()[1]["name"]
bronze_external_location_name = "bronze_storage_location"
bronze_url = "abfss://databricks@djsdatabricksstor.dfs.core.windows.net/bronze"

In [0]:
location_query = """
CREATE EXTERNAL LOCATION IF NOT EXISTS `{0}`
URL '{1}'
WITH (STORAGE CREDENTIAL `{2}`);
""".format(bronze_external_location_name,bronze_url,unity_storage_credential)

spark.sql(location_query)


In [0]:
volume_location = "abfss://databricks@djsdatabricksstor.dfs.core.windows.net/bronze"
volume_name ="main.djsprojects.bronze"
volume_query = """
CREATE EXTERNAL VOLUME IF NOT EXISTS {0}
       LOCATION '{1}'
""".format(volume_name, volume_location)

spark.sql(volume_query)

In [0]:
download_path = '/Volumes/{0}'.format(volume_name.replace('.','/'))

In [0]:
url = 'https://datasets.imdbws.com/'
html_content = requests.get(url).text
# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the list by its ID
items_list = soup.find('ul')
for item in items_list.findAll('a'):
   file_name =item.getText()
   decompressed_file_name = file_name.replace('.gz', '')
   file_path = item.get('href')
   dest_download_path = "/tmp/{}".format(file_name)
   urllib.request.urlretrieve(file_path, dest_download_path)
   os.system('gzip -d {}'.format(dest_download_path))
   os.system("cp /tmp/{0} {1}".format(decompressed_file_name, download_path))

In [0]:
for file in dbutils.fs.ls(download_path):
    table_name = file.name.replace('.tsv', '').replace('.', '_')
    path = file.path
    df = spark.read.option("delimiter", "\t").option("header", "true").csv(path)
    df.write.saveAsTable("main.djsprojects.{0}".format(table_name))

In [0]:
%sql
SELECT
  tb.startYear,
  tb.genres,
  tr.averageRating 
FROM main.djsprojects.title_basics AS tb
INNER JOIN main.djsprojects.title_ratings AS tr ON tb.tconst = tr.tconst 
WHERE tb.titleType = 'movie'
GROUP BY tb.genres,tr.averageRating,tb.startYear
ORDER BY tr.averageRating DESC
LIMIT 10
